In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"D:\Generative AI\generative ai project\QA_SQL_Tabular_data\data\for_uploads\titanic_small.csv")

In [3]:
df.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35,0,0,8.0500


In [5]:
import os
import pandas as pd
import chromadb
from langchain_community.llms import Ollama
from sentence_transformers import SentenceTransformer

# Initialize the Ollama model
llama3 = Ollama(model="llama3")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# chroma is already created

In [6]:
# Perform similarity search

query_texts = "what's the average age of survivors"

query_embeddings = embedding_model.encode(query_texts)

In [13]:
type(query_embeddings.tolist())

list

In [8]:
chroma_client = chromadb.PersistentClient(path=r"D:\Generative AI\generative ai project\QA_SQL_Tabular_data\data\chroma")
vectordb = chroma_client.get_collection(name="titanic_data")
vectordb.count()

30

In [14]:
results = vectordb.query(
    query_embeddings = query_embeddings.tolist(),
    n_results=1 #top_k
)

results

{'ids': [['id11']],
 'distances': [[1.2230411319537413]],
 'metadatas': [[{'source': 'titanic_small'}]],
 'embeddings': None,
 'documents': [['Survived: 1,\nPclass: 1,\nName: Miss. Elizabeth Bonnell,\nSex: female,\nAge: 58,\nSiblings/Spouses Aboard: 0,\nParents/Children Aboard: 0,\nFare: 26.55,\n']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [15]:
system_role = "You will recieve the user's question along with the search results of that question over a database. Give the user the proper answer."
prompt = f"User's question: {query_texts} \n\n Search results:\n {results}"

messages = [
    {"role": "system", "content": str(
        system_role
        )},
    {"role": "user", "content": prompt}
]

In [16]:
response = llama3.invoke(messages)

In [17]:
response

'According to the search results, the average age of survivors is not provided in the given database snippet. However, we do have information about one survivor: Miss. Elizabeth Bonnell, who was 58 years old.\n\nIf you would like to know more about the average age of survivors on the Titanic or have additional questions, please feel free to ask!'